### Install & import

In [ ]:
%pip install langchain
%pip install openai

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 817.3/817.3 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.0/90.0 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 1.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 kB 463.5 kB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.9/71.9 kB 2.8 MB/s eta 0:00:00


In [ ]:
import pandas as pd
from langchain.llms import OpenAI
from langchain.agents import create_csv_agent, create_pandas_dataframe_agent
from langchain.memory import ConversationBufferMemory, ReadOnlySharedMemory

### OpenAI login

In [ ]:
import getpass
pswd = getpass.getpass('OpenAI API key:')

OpenAI API key:··········


### Load CSV files

In [ ]:
# template.csv
from google.colab import files
upload = files.upload()

Saving template.csv to template.csv


In [ ]:
# table_A.csv
from google.colab import files
upload = files.upload()

Saving table_A.csv to table_A.csv


In [ ]:
# table_B.csv
from google.colab import files
upload = files.upload()

Saving table_B.csv to table_B.csv


### *Part 1* Map table A column to template's columns and save resulting DF

In [ ]:
agent = create_csv_agent(OpenAI(temperature=0, openai_api_key=pswd, model_name="text-davinci-003"), 'table_A.csv', verbose=True)

In [ ]:
mapping_to_template = agent.run(f"Load template_df with template_df = pd.read_csv('template.csv'). \
Compute sequence similarity between all possible pairs of columns from two dataframes. \
Pair columns when ratios when ratio is positive.")
mapping_to_template



> Entering new AgentExecutor chain...

Thought: I need to compare the columns of two dataframes.
Action: python_repl_ast
Action Input: from difflib import SequenceMatcher
Observation: 
Thought: I need to compare the columns of two dataframes.
Action: python_repl_ast
Action Input: template_df = pd.read_csv('template.csv')
Observation: NameError("name 'pd' is not defined")
Thought: I need to import pandas
Action: python_repl_ast
Action Input: import pandas as pd
Observation: 
Thought: I need to read the csv file
Action: python_repl_ast
Action Input: template_df = pd.read_csv('template.csv')
Observation: 
Thought: I need to compare the columns of two dataframes.
Action: python_repl_ast
Action Input: for col1 in df.columns:
    for col2 in template_df.columns:
        ratio = SequenceMatcher(None, df[col1], template_df[col2]).ratio()
        if ratio > 0:
            print(f'{col1} and {col2} have a similarity ratio of {ratio}')
Observation: FullName and EmployeeName have a similarity ra

'The columns FullName and EmployeeName, Monthly_Premium and Premium, Full_Name and EmployeeName, Insurance_Type and Plan, and Monthly_Cost and Premium have a similarity ratio of 1.0.'

In [ ]:
mapping_json = agent.run(f"{mapping_to_template}. Use this info to create JSON with column mapping. Print this JSON").replace('\'','"')
mapping_json



> Entering new AgentExecutor chain...
Thought: I need to use the dataframe to create the JSON
Action: python_repl_ast
Action Input: df.columns.to_json()
Observation: AttributeError("'Index' object has no attribute 'to_json'")
Thought: I need to convert the dataframe columns to a list
Action: python_repl_ast
Action Input: list(df.columns)
Observation: ['Date_of_Policy', 'FullName', 'Insurance_Plan', 'Policy_No', 'Monthly_Premium', 'Department', 'JobTitle', 'Policy_Start', 'Full_Name', 'Insurance_Type', 'Policy_Num', 'Monthly_Cost']
Thought: I need to create a dictionary with the column mappings
Action: python_repl_ast
Action Input: {'FullName': 'EmployeeName', 'Monthly_Premium': 'Premium', 'Full_Name': 'EmployeeName', 'Insurance_Type': 'Plan', 'Monthly_Cost': 'Premium'}
Observation: {'FullName': 'EmployeeName', 'Monthly_Premium': 'Premium', 'Full_Name': 'EmployeeName', 'Insurance_Type': 'Plan', 'Monthly_Cost': 'Premium'}
Thought: I need to convert the dictionary to JSON
Action: python

'{"FullName": "EmployeeName", "Monthly_Premium": "Premium", "Full_Name": "EmployeeName", "Insurance_Type": "Plan", "Monthly_Cost": "Premium"}'

In [ ]:
import json
template_df = pd.read_csv('template.csv')
not_matched = list(template_df.columns.difference(json.loads(mapping_json).values()))
not_matched

['Date', 'PolicyNumber']

In [ ]:
# mapping_missing = agent.run(f"Find columns in df best matching to {not_matched}. Print as map from df to new columns as a JSON").replace('\'','"')
matched_to = agent.run(f"Find columns in df best matching to {not_matched}. Print as a list").replace('\'','"')
mapping_missing = dict(zip(json.loads(matched_to), not_matched))
mapping_missing



> Entering new AgentExecutor chain...
Thought: I need to find the columns that match the given list
Action: python_repl_ast
Action Input: df.columns.tolist()
Observation: ['Date_of_Policy', 'FullName', 'Insurance_Plan', 'Policy_No', 'Monthly_Premium', 'Department', 'JobTitle', 'Policy_Start', 'Full_Name', 'Insurance_Type', 'Policy_Num', 'Monthly_Cost']
Thought: I need to filter the list to only include the columns I need
Action: python_repl_ast
Action Input: [col for col in df.columns.tolist() if col in ['Date', 'PolicyNumber']]
Observation: []
Thought: I need to use the correct column names
Action: python_repl_ast
Action Input: [col for col in df.columns.tolist() if col in ['Date_of_Policy', 'Policy_No']]
Observation: ['Date_of_Policy', 'Policy_No']
Thought: I now know the final answer
Final Answer: ['Date_of_Policy', 'Policy_No']

> Finished chain.


{'Date_of_Policy': 'Date', 'Policy_No': 'PolicyNumber'}

In [ ]:
_columns = json.loads(mapping_json)
_columns.update(mapping_missing)
_columns

{'FullName': 'EmployeeName',
 'Monthly_Premium': 'Premium',
 'Full_Name': 'EmployeeName',
 'Insurance_Type': 'Plan',
 'Monthly_Cost': 'Premium',
 'Date_of_Policy': 'Date',
 'Policy_No': 'PolicyNumber'}

In [ ]:
# apply mapping_code to `table_A.csv`
df = pd.read_csv("table_A.csv")
df.rename(columns=_columns, inplace=True)
df = df[pd.read_csv("template.csv").columns]
# Keep the first occurrence of each duplicated column
df = df.loc[:, ~df.columns.duplicated(keep='first')]
df['Date'] = pd.to_datetime(df['Date'])
df

,Date,EmployeeName,Plan,PolicyNumber,Premium
0,2023-05-01,John Doe,Gold,AB-12345,150.0
1,2023-05-02,Jane Smith,Silver,CD-67890,100.0
2,2023-05-03,Michael Brown,Bronze,EF-10111,50.0
3,2023-05-04,Alice Johnson,Gold,GH-12121,150.0
4,2023-05-05,Bob Wilson,Silver,IJ-13131,100.0
5,2023-05-06,Carol Martinez,Bronze,KL-14141,50.0
6,2023-05-07,David Anderson,Gold,MN-15151,150.0
7,2023-05-08,Eva Thomas,Silver,OP-16161,100.0
8,2023-05-09,Frank Jackson,Bronze,QR-17171,50.0
9,2023-05-10,Grace White,Gold,ST-18181,150.0


### *Part 2* Align the column formatting of the DF with that of template_df

In [ ]:
agent = create_pandas_dataframe_agent(OpenAI(temperature=0, openai_api_key=pswd, model_name="text-davinci-003"), df, verbose=True)

In [ ]:
template_df = pd.read_csv('template.csv')
col_transform_code = agent.run(f"Make the formatting of every columns match the correcponding column in {template_df}. Output the code")
col_transform_code



> Entering new AgentExecutor chain...

Thought: I need to change the format of the date column
Action: python_repl_ast
Action Input: df['Date'] = df['Date'].dt.strftime('%d-%m-%Y')
Observation: 
Thought: I need to change the format of the PolicyNumber column
Action: python_repl_ast
Action Input: df['PolicyNumber'] = df['PolicyNumber'].str.replace('-', '')
Observation: 
Thought: I now know the final answer
Final Answer: df['Date'] = df['Date'].dt.strftime('%d-%m-%Y')
df['PolicyNumber'] = df['PolicyNumber'].str.replace('-', '')

> Finished chain.


"df['Date'] = df['Date'].dt.strftime('%d-%m-%Y')\ndf['PolicyNumber'] = df['PolicyNumber'].str.replace('-', '')"

In [ ]:
# apply mapping_code and col_transform_code to `table_A.csv`
df = pd.read_csv("table_A.csv")
df.rename(columns=_columns, inplace=True)
df = df[pd.read_csv("template.csv").columns]
# Keep the first occurrence of each duplicated column
df = df.loc[:, ~df.columns.duplicated(keep='first')]
df['Date'] = pd.to_datetime(df['Date'])
exec(col_transform_code)
df

,Date,EmployeeName,Plan,PolicyNumber,Premium
0,01-05-2023,John Doe,Gold,AB12345,150.0
1,02-05-2023,Jane Smith,Silver,CD67890,100.0
2,03-05-2023,Michael Brown,Bronze,EF10111,50.0
3,04-05-2023,Alice Johnson,Gold,GH12121,150.0
4,05-05-2023,Bob Wilson,Silver,IJ13131,100.0
5,06-05-2023,Carol Martinez,Bronze,KL14141,50.0
6,07-05-2023,David Anderson,Gold,MN15151,150.0
7,08-05-2023,Eva Thomas,Silver,OP16161,100.0
8,09-05-2023,Frank Jackson,Bronze,QR17171,50.0
9,10-05-2023,Grace White,Gold,ST18181,150.0


### Do the same for table_B

In [ ]:
agent = create_csv_agent(OpenAI(temperature=0, openai_api_key=pswd, model_name="text-davinci-003"), 'table_B.csv', verbose=True)

In [ ]:
mapping_to_template = agent.run(f"Load template_df with template_df = pd.read_csv('template.csv'). \
Compute sequence similarity between all possible pairs of columns from two dataframes. \
Pair columns when ratios when ratio is positive.")
mapping_to_template



> Entering new AgentExecutor chain...
Thought: I need to compare the columns of two dataframes
Action: python_repl_ast
Action Input: from difflib import SequenceMatcher
Observation: 
Thought: I need to loop through the columns of both dataframes
Action: python_repl_ast
Action Input: for col1 in df.columns:
    for col2 in template_df.columns:
        ratio = SequenceMatcher(None, df[col1], template_df[col2]).ratio()
        if ratio > 0:
            print(f'{col1} and {col2} have a similarity ratio of {ratio}')
Observation: NameError("name 'template_df' is not defined")
Thought: I need to load the template_df
Action: python_repl_ast
Action Input: template_df = pd.read_csv('template.csv')
Observation: NameError("name 'pd' is not defined")
Thought: I need to import pandas
Action: python_repl_ast
Action Input: import pandas as pd
Observation: 
Thought: I need to load the template_df
Action: python_repl_ast
Action Input: template_df = pd.read_csv('template.csv')
Observation: 
Thought: I 

'The columns PremiumAmount and PolicyNumber, Employee_Name and EmployeeName, Plan_Name and Plan, PolicyID and PolicyNumber, and Cost and Premium have a similarity ratio of 1.0.'

In [ ]:
mapping_json = agent.run(f"{mapping_to_template}. Use this info to create JSON with column mapping. Print this JSON").replace('\'','"')
mapping_json



> Entering new AgentExecutor chain...
Thought: I need to find the column names and create a JSON
Action: python_repl_ast
Action Input: df.columns
Observation: Index(['PolicyDate', 'Name', 'PlanType', 'Policy_ID', 'PremiumAmount', 'Hobby',
       'MaritalStatus', 'StartDate', 'Employee_Name', 'Plan_Name', 'PolicyID',
       'Cost'],
      dtype='object')
Thought: I need to create a dictionary with the column mappings
Action: python_repl_ast
Action Input: 
column_mapping = {
    'PremiumAmount': 'PolicyNumber',
    'Employee_Name': 'EmployeeName',
    'Plan_Name': 'Plan',
    'PolicyID': 'PolicyNumber',
    'Cost': 'Premium'
}
Observation: 
Thought: I need to convert the dictionary to JSON
Action: python_repl_ast
Action Input: 
import json

column_mapping_json = json.dumps(column_mapping)
Observation: 
Thought: I need to print the JSON
Action: python_repl_ast
Action Input: print(column_mapping_json){"PremiumAmount": "PolicyNumber", "Employee_Name": "EmployeeName", "Plan_Name": "Plan", 

'{"PremiumAmount": "PolicyNumber", "Employee_Name": "EmployeeName", "Plan_Name": "Plan", "PolicyID": "PolicyNumber", "Cost": "Premium"}'

In [ ]:
import json
template_df = pd.read_csv('template.csv')
not_matched = list(template_df.columns.difference(json.loads(mapping_json).values()))
not_matched

['Date']

In [ ]:
# mapping_missing = agent.run(f"Find columns in df best matching to {not_matched}. Print as map from df to new columns as a JSON").replace('\'','"')
matched_to = agent.run(f"Find columns in df best matching to {not_matched}. Print as a list").replace('\'','"')
mapping_missing = dict(zip(json.loads(matched_to), not_matched))
mapping_missing



> Entering new AgentExecutor chain...
Thought: I need to find the columns that have the word 'Date' in them
Action: python_repl_ast
Action Input: df.columns[df.columns.str.contains('Date')].tolist()
Observation: ['PolicyDate', 'StartDate']
Thought: I now know the final answer
Final Answer: ['PolicyDate', 'StartDate']

> Finished chain.


{'PolicyDate': 'Date'}

In [ ]:
_columns = json.loads(mapping_json)
_columns.update(mapping_missing)
_columns

{'PremiumAmount': 'PolicyNumber',
 'Employee_Name': 'EmployeeName',
 'Plan_Name': 'Plan',
 'PolicyID': 'PolicyNumber',
 'Cost': 'Premium',
 'PolicyDate': 'Date'}

In [ ]:
# apply mapping_code to `table_B.csv`
df = pd.read_csv("table_B.csv")
df.rename(columns=_columns, inplace=True)
df = df[pd.read_csv("template.csv").columns]
# Keep the first occurrence of each duplicated column
df = df.loc[:, ~df.columns.duplicated(keep='first')]
df['Date'] = pd.to_datetime(df['Date'])
df

,Date,EmployeeName,Plan,PolicyNumber,Premium
0,2023-05-01,John Doe,Gold,AB12345,150
1,2023-05-02,Jane Smith,Silver,CD67890,100
2,2023-05-03,Michael Brown,Bronze,EF10111,50
3,2023-05-04,Alice Johnson,Gold,GH12121,150
4,2023-05-05,Bob Wilson,Silver,IJ13131,100
5,2023-05-06,Carol Martinez,Bronze,KL14141,50
6,2023-05-07,David Anderson,Gold,MN15151,150
7,2023-05-08,Eva Thomas,Silver,OP16161,100
8,2023-05-09,Frank Jackson,Bronze,QR17171,50
9,2023-05-10,Grace White,Gold,ST18181,150


In [ ]:
agent = create_pandas_dataframe_agent(OpenAI(temperature=0, openai_api_key=pswd, model_name="text-davinci-003"), df, verbose=True)

In [ ]:
template_df = pd.read_csv('template.csv')
col_transform_code = agent.run(f"Make the formatting of every columns match the correcponding column in {template_df}. Output the code")
col_transform_code



> Entering new AgentExecutor chain...

Thought: I need to change the format of the date column
Action: python_repl_ast
Action Input: df['Date'] = df['Date'].dt.strftime('%d-%m-%Y')
Observation: 
Thought: I need to output the dataframe
Action: python_repl_ast
Action Input: print(df)         Date    EmployeeName    Plan PolicyNumber  Premium
0  01-05-2023        John Doe    Gold      AB12345      150
1  02-05-2023      Jane Smith  Silver      CD67890      100
2  03-05-2023   Michael Brown  Bronze      EF10111       50
3  04-05-2023   Alice Johnson    Gold      GH12121      150
4  05-05-2023      Bob Wilson  Silver      IJ13131      100
5  06-05-2023  Carol Martinez  Bronze      KL14141       50
6  07-05-2023  David Anderson    Gold      MN15151      150
7  08-05-2023      Eva Thomas  Silver      OP16161      100
8  09-05-2023   Frank Jackson  Bronze      QR17171       50
9  10-05-2023     Grace White    Gold      ST18181      150

Observation: None
Thought: I now know the final answer


"df['Date'] = df['Date'].dt.strftime('%d-%m-%Y')\nprint(df)"

In [ ]:
# apply mapping_code and col_transform_code to `table_B.csv`
df = pd.read_csv("table_B.csv")
df.rename(columns=_columns, inplace=True)
df = df[pd.read_csv("template.csv").columns]
# Keep the first occurrence of each duplicated column
df = df.loc[:, ~df.columns.duplicated(keep='first')]
df['Date'] = pd.to_datetime(df['Date'])
exec(col_transform_code)
df

         Date    EmployeeName    Plan PolicyNumber  Premium
0  01-05-2023        John Doe    Gold      AB12345      150
1  02-05-2023      Jane Smith  Silver      CD67890      100
2  03-05-2023   Michael Brown  Bronze      EF10111       50
3  04-05-2023   Alice Johnson    Gold      GH12121      150
4  05-05-2023      Bob Wilson  Silver      IJ13131      100
5  06-05-2023  Carol Martinez  Bronze      KL14141       50
6  07-05-2023  David Anderson    Gold      MN15151      150
7  08-05-2023      Eva Thomas  Silver      OP16161      100
8  09-05-2023   Frank Jackson  Bronze      QR17171       50
9  10-05-2023     Grace White    Gold      ST18181      150


,Date,EmployeeName,Plan,PolicyNumber,Premium
0,01-05-2023,John Doe,Gold,AB12345,150
1,02-05-2023,Jane Smith,Silver,CD67890,100
2,03-05-2023,Michael Brown,Bronze,EF10111,50
3,04-05-2023,Alice Johnson,Gold,GH12121,150
4,05-05-2023,Bob Wilson,Silver,IJ13131,100
5,06-05-2023,Carol Martinez,Bronze,KL14141,50
6,07-05-2023,David Anderson,Gold,MN15151,150
7,08-05-2023,Eva Thomas,Silver,OP16161,100
8,09-05-2023,Frank Jackson,Bronze,QR17171,50
9,10-05-2023,Grace White,Gold,ST18181,150
